In [23]:
import numpy as np
from numpy.testing import assert_allclose

# Part I. Construct a Householder reflection of a vector.

Given a vector $\mathbf{x}$, and a plane with a normal vector $\mathbf{u}$, the Householder transformation reflects $\mathbf{x}$ about the plane.

The matrix of the Householder transformation is

$$
\mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T
$$

Given two equal-length vectors $\mathbf{x}$ and $\mathbf{y}$, a rotation which brings $\mathbf{x}$ to $\mathbf{y}$ is a Householder transform with

$$
\mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{\left|\mathbf{x} - \mathbf{y}\right|}
$$

Write a function which rotates a given vector, $\mathbf{x} = (x_1, \dots, x_n)$ into $\mathbf{y} = (\left|\mathbf{x}\right|, 0, \dots, 0)^T$ using a Householder transformation.

In [16]:
def householder(vector):
    """Construct a Householder reflection to zero out 2nd and further components of a vector.

    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Input vector
    
    Returns
    -------
    outvec : array of floats, shape (n,)
        Transformed vector, with ``outvec[1:]==0`` and ``|outvec| == |vec|``
    H : array of floats, shape (n, n)
        Orthogonal matrix of the Householder reflection
    """
    vector = np.asarray(vector, dtype=float)
    if vector.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vector.ndim)
    
    y = np.zeros_like(vector)
    y[0] = np.linalg.norm(vector) 
    u = (vector - y) / np.linalg.norm(vector - y)
    H = np.eye(u.shape[0]) - 2 * np.outer(u, u)
    
    return y, H

Test your function using tests below:

In [17]:
# Test I.1 (10% of the total grade).

v = np.array([1, 2, 3])
v1, h = householder(v)

assert_allclose(np.dot(h, v1), v)
assert_allclose(np.dot(h, v), v1)

AssertionError: 
Not equal to tolerance rtol=1e-07, atol=0

Mismatched elements: 1 / 3 (33.3%)
Max absolute difference: 0.
Max relative difference: 0.
 x: array([ 3.741657,  0.      , -0.      ])
 y: array([3.741657, 0.      , 0.      ])

In [18]:
# Test I.2 (10% of the total grade).

rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)

assert_allclose(np.dot(h, v1), vec)

# Part II. Compute the $\mathrm{QR}$ decomposition of a matrix.

Given a rectangular $m\times n$ matrix $\mathbf{A}$, construct a Householder reflector matrix $\mathbf{H}_1$ which transforms the first column of $\mathbf{A}$ (and call the result $\mathbf{A}^{(1)}$)

$$
\mathbf{H}_1 \mathbf{A} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
&& \dots&& \\
0      & \times & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(1)}\;.
$$

Now consider the lower-right submatrix of $\mathbf{A}^{(1)}$, and construct a Householder reflector which annihilates the second column of $\mathbf{A}$:

$$
\mathbf{H}_2 \mathbf{A}^{(1)} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & 0      & \times & \dots & \times \\
&& \dots&& \\
0      & 0      & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(2)} \;.
$$

Repeating the process $n-1$ times, we obtain

$$
\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;,
$$

with $\mathbf{R}$ an upper triangular matrix. Since each $\mathbf{H}_k$ is orthogonal, so is their product. The inverse of an orthogonal matrix is orthogonal. Hence the process generates the $\mathrm{QR}$ decomposition of $\mathbf{A}$. 

Write a function, which receives a recangular matrix, $A$, and returns the Q and R factors of the $QR$ factorization of $A$.

In [19]:
def qr_decomp(a):
    """Compute the QR decomposition of a matrix.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    q : ndarray, shape(m, m)
        The orthogonal matrix
    r : ndarray, shape(m, n)
        The upper triangular matrix
        
    Examples
    --------
    >>> a = np.random.random(size=(3, 5))
    >>> q, r = qr_decomp(a)
    >>> np.assert_allclose(np.dot(q, r), a)
    
    """
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    H = np.eye(m)
    
    for i in range(min(n, m)):
        h = householder(a1[i:, i])[1]
        H_i = np.eye(m)
        H_i[i:, i:] = h
        a1 = H_i @ a1
        H = H_i @ H
    Q = H.T
    
    return Q, a1

In [20]:
# Might want to turn this on for prettier printing: zeros instead of `1e-16` etc

np.set_printoptions(suppress=True)

In [21]:
# Test II.1 (20% of the total grade)

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
q, r = qr_decomp(a)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)

Now compare your decompositions to the library function (which actually wraps the corresponding LAPACK functions)

In [22]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)

Check if your `q` and `r` agree with `qq` and `rr`. Explain.

*Enter your explanation here* (10% of the total grade, peer-graded)

# Part III. Avoid forming Householder matrices explicitly.

Note the special structure of the Householder matrices: A reflector $\mathbf{H}$ is completely specified by a reflection vector $\mathbf{u}$. Also note that the computational cost of applying a reflector to a matrix strongly depends on the order of operations:

$$
\left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A}  \qquad \textrm{is } O(m^2 n)\;,
$$
while
$$
\mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn)
$$

Thus, it seems to make sense to *avoid* forming the $\mathbf{H}$ matrices. Instead, one stores the reflection vectors, $\mathbf{u}$, themselves, and provides a way of multiplying an arbitrary matrix by $\mathbf{Q}^T$, e.g., as a standalone function (or a class).

Write a function which constructs the `QR` decomposition of a matrix *without ever forming the* $\mathbf{H}$ matrices, and returns the $\mathbf{R}$ matrix and reflection vectors. 

Write a second function, which uses reflection vectors to multiply a matrix with $\mathbf{Q}^T$. Make sure to include enough comments for a marker to follow your implementation, and add tests. 

(Peer-graded, 40% of the total grade)

In [39]:
def householder(vec):
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    y = np.zeros_like(vec)
    y[0] = np.linalg.norm(vec)
    u = (vec - y) / np.linalg.norm(vec - y)
    u = np.reshape(u, (len(u),1))
    return y, u


def qr_decomp_1_2(A): 
    m, n = A.shape
    U=np.eye(m)
    R = np.array(A, copy=True, dtype=float)
    R_u = np.array(A, copy=True, dtype=float)
    x = A[0:,0]
    y, u = householder(x)
    u_T=np.reshape(u, (1,len(u)))
    R_u = u.T @ R_u
    R_u = 2*u @ R_u
    R = R - R_u 
    U[0:,0] = u_T 
    for i in range(1,min(m-1,n)):
        R_i = R[i:,i:]
        Riu = R[i:,i:]
        x = R_i[0:, 0]
        y, u = householder(x)
        u_T=np.reshape(u, (1,len(u))) 
        Riu = u_T @ Riu
        Riu = 2*u @ Riu
        R_i = R_i - Riu
        U[i:,i]=u_T 
        R[i:,i:] = R_i
    return R, U

def qr_decomp_2_1(U,A): 
    m, n = A.shape
    R = np.array(A, copy=True, dtype=float)
    R_u = np.array(A, copy=True, dtype=float)
    u_T=U[0:,0]
    u=np.reshape(u_T, (len(u_T),1))
    R_u = u.T @ R_u
    R_u = 2*u @ R_u
    R = R - R_u 
    U[0:,0] = u_T
    for i in range(1,min(m-1,n)):
        Ri = R[i:,i:]
        Riu = R[i:,i:]
        u_T=U[i:,i]  
        u=np.reshape(u_T, (len(u_T),1))
        u_T=np.reshape(u, (1,len(u_T)))
        Riu = u_T @ Riu
        Riu = 2*u @ Riu
        Ri = Ri - Riu
        U[i:,i]=u_T
        R[i:,i:] = Ri
    return R

r, u = qr_decomp_1_2(a)
q = qr_decomp_2_1(u, a)

assert_allclose(np.dot(q, q.T), np.eye(q.shape[0]), atol=1e-10)
assert_allclose(np.dot(q, r), a)

AssertionError: 
Not equal to tolerance rtol=1e-07, atol=1e-10

Mismatched elements: 11 / 25 (44%)
Max absolute difference: 3.33182443
Max relative difference: 3.33182443
 x: array([[ 4.331824,  1.665959,  0.492026,  0.      ,  0.      ],
       [ 1.665959,  1.176773,  0.376193,  0.      , -0.      ],
       [ 0.492026,  0.376193,  0.302065,  0.      , -0.      ],...
 y: array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],...